# __Segmenting and Clustering Neighbourhoods in Toronto__
This notebooks is used for the Week 3 assignment within the Courser course 'Applied Data Science Capstone'

## Part 1
We start with extracting the data from the wikipedia page <br/>
To replace 'Not assigned' with NaN we set up the parameter 'na_values'. That will help us to modify the dataset later.

In [1]:
# Let's start with extracting the data from the Wikipedia page

import pandas as pd
import numpy as np

# We set the parameter 'na_values' so to replase the values 'Not assigned' with NaN
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', na_values = 'Not assigned')[0]
data.head()

,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Now we have to format and prepare data. To do it:
> We remove all the rows where 'Borough' is NaN <br/>
> We replace NaN in 'Neighbourhoods' column with values from 'Borough' one

In [2]:
# Let's remove all the rows where 'Borough' is NaN
data.dropna(axis=0, subset=['Borough'], inplace=True)

# Let's replace NaN in Neighbourhoods with Borough values
data.fillna(axis=1, method='ffill', inplace=True)

Now we have to combine rows with the same Postocode and aggregate their Neighbourhood with comma separator

In [3]:
data1=data.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join)

To present the results as DataFrame of a proper format we will reset both levels of the index. <br/>
To be shure that everything is OK let's look at the first 30 rows and look for the indexes on the screenshot provided as an example.

In [4]:
data1=pd.DataFrame(data1)
data1.reset_index(level=1, inplace=True)
data1.reset_index(level=0, inplace=True)

# Let's look at the first 30 rows and compare some of them with the screenshot provided
data1.head(30)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


And finally we have to print the number of the rows with .shape method

In [5]:
print("The number of rows of the final dataframe: ", data1.shape[0])

The number of rows of the final dataframe:  103
